In [1]:
from pycodex.segmentation import run_segmentation_mesmer_cell
from pycodex.io import setup_gpu
from pathlib import Path

setup_gpu("1")

Using GPU(s): ['/physical_device:GPU:0']


In [2]:
unit_dir = Path("../demo/data/segmentation/reg001/")
!tree {unit_dir}

../demo/data/segmentation/reg001
└── reg001.ome.tiff

0 directories, 1 file


In [3]:
internal_markers = ["DAPI"]
boundary_markers = ["CD45", "CD3e", "CD163", "NaKATP"]
thresh_q_min = 0
thresh_q_max = 0.99
thresh_otsu = False
scale = True
pixel_size_um = 0.5068164319979996
maxima_threshold = 0.075
interior_threshold = 0.20

tag = None
run_segmentation_mesmer_cell(
    unit_dir=unit_dir,
    internal_markers=internal_markers,
    boundary_markers=boundary_markers,
    thresh_q_min=thresh_q_min,
    thresh_q_max=thresh_q_max,
    thresh_otsu=thresh_otsu,
    scale=scale,
    pixel_size_um=pixel_size_um,
    maxima_threshold=maxima_threshold,
    interior_threshold=interior_threshold,
    tag=tag,
)

INFO:root:OME-TIFF file loaded: ../demo/data/segmentation/reg001/reg001.ome.tiff.



Logging is configured. Logs are saved to: ../demo/data/segmentation/reg001/20250530_010550/segmentation.log and displayed in the console.


2025-05-30 01:05:52.229335: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-30 01:05:52.733346: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22221 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:4b:00.0, compute capability: 8.6


2025-05-30 01:05:57,694 - WARNING - No training configuration found in save file, so the model was *not* compiled. Compile it manually.
2025-05-30 01:05:59.320397: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2025-05-30 01:05:59.992541: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
INFO:root:Segmentation completed.
Writing tiles: 100%|██████████| 21/21 [00:00<00:00, 216.83it/s]
INFO:root:Markers used for segmentation saved as OME-TIFF.
INFO:root:Single-cell features extracted.


In [4]:
!tree {unit_dir}

../demo/data/segmentation/reg001
├── 20250530_010550
│   ├── data.csv
│   ├── dataScaleSize.csv
│   ├── parameter_segmentation.json
│   ├── segmentation.log
│   ├── segmentation_markers.ome.tiff
│   └── segmentation_mask.tiff
└── reg001.ome.tiff

1 directory, 7 files
